In [1]:
from rdkit import Chem
from rdkit.Chem import RDConfig
from rdkit.Chem.QED import qed
import os
from tqdm import tqdm
import seaborn as sns
import numpy as np

In [2]:
import sys
sys.path.append('..')
sys.path.append(os.path.join(RDConfig.RDContribDir, "SA_Score"))
import sascorer

In [3]:
from utils_eval import build_pdb_dict


In [17]:
# root_path = '../evaluation/ligand_based'
root_path = '../evaluation/structure_based'
bridge_type = 'vp'

# ligand_name = '3hfg_A_rec_3hfg_17r_lig_tt_min_0'
# ligand_name = '3ey4_C_rec_5pgu_8k4_lig_tt_min_0'  # 0.11 0.10
# ligand_name = '5ni2_A_rec_5aen_dp8_lig_tt_min_0'
# ligand_name = '3fu5_A_rec_5ni6_dj3_lig_tt_min_0'
# ligand_name = '4yyz_B_rec_2ilt_nn1_lig_tt_min_0'  # 0.16 0.20 
# ligand_name = '4bnn_A_rec_4bnm_tcu_lig_tt_min_0'
# ligand_name = '4yvx_A_rec_1s2a_imn_lig_tt_min_0'
# ligand_name = '3czr_B_rec_4bb5_hd2_lig_tt_min_0'
# ligand_name = '3czr_A_rec_2ilt_nn1_lig_tt_min_0'
# ligand_name = '4b84_A_rec_4btl_5gz_lig_tt_min_0'
# ligand_name = '3fue_A_rec_3ftv_11x_lig_tt_min_0'  # 0.04 0.05
# ligand_name = '4bnk_B_rec_4d44_ja3_lig_tt_min_0'  # 0.26 0.27
# ligand_name = '3qqp_B_rec_4k1l_sff_lig_tt_min_0'
# ligand_name = '4wdu_A_rec_3r94_flr_lig_tt_min_0'  # 0.37 0.27
# ligand_name = '4alj_A_rec_4bnf_pv4_lig_tt_docked_0'  # 0.24 0.24
# ligand_name = '1ejj_A_rec_1o99_2pg_lig_tt_docked_0'
# ligand_name = '4h96_B_rec_4h95_06u_lig_tt_min_0'  # 0.40 0.42
# ligand_name = '2oa6_D_rec_4kvi_1sv_lig_tt_min_0'  # 0.36 0.41
# ligand_name = '4hx5_B_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgy_D_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '5pgv_A_rec_3bzu_a21_lig_tt_min_0'
# ligand_name = '4kux_B_rec_4kvw_jf4_lig_tt_min_0'  # 0.61 0.63
# ligand_name = '4bnk_B_rec_4bnl_w1g_lig_tt_min_0'
# ligand_name = '4b82_B_rec_4ara_c56_lig_tt_min_0'  # 0.42 0.53
# ligand_name = '4pm0_A_rec_4y2b_epk_lig_tt_min_0'
# ligand_name = '4ms6_A_rec_3fuh_5h1_lig_tt_min_0'  # 0.02 0.02
# ligand_name = '5ivg_B_rec_4kvy_jf1_lig_tt_docked_9'  # 0.61 0.69
# ligand_name = '3r8h_A_rec_3r8h_zom_lig_tt_min_0'
# ligand_name = '1aoe_A_rec_3qlr_qlr_lig_tt_docked_4'  # 0.19 0.28
# ligand_name = '2ilt_A_rec_3ey4_352_lig_tt_min_0'  # 0.70 0.71
# ligand_name = '2g1q_A_rec_1x88_nat_lig_tt_min_0'  # 0.59 0.59
# ligand_name = '5imp_B_rec_4kwd_jf2_lig_tt_min_0'  # 0.54 0.55
# ligand_name = '4d45_A_rec_4ali_tcl_lig_tt_min_0'  # 0.57 0.52
# ligand_name = '5ivg_B_rec_4kvw_jf4_lig_tt_min_0'
# ligand_name = '2rbe_C_rec_3byz_h11_lig_tt_min_0'
# ligand_name = '2whr_A_rec_3zlv_hi6_lig_tt_min_0'
# ligand_name = '4hmn_A_rec_3ufy_npx_lig_tt_min_0'
ligand_name = '4h7c_A_rec_3r6i_jms_lig_tt_min_0'

aromatic = True

gen_path = os.path.join(root_path, bridge_type, ligand_name, 'aromatic' if aromatic else 'basic')

In [18]:
raw_data_path = '../../data/cleaned_crossdocked_data/raw'
pdb_dict, pdb_rev_dict = build_pdb_dict(raw_data_path)

In [19]:
def get_mols(gen_path, ligand_name=None, raw_data_path=raw_data_path):
    gen_mols, ref_mols = {}, {}
    for file in tqdm(os.listdir(gen_path)):
        ligand = file.split('.')[0]
        ref_folder = pdb_rev_dict[ligand_name+'.sdf']
        # ref_folder = pdb_rev_dict[file]

        gen_m = Chem.MolFromMolFile(os.path.join(gen_path, file))
        ref_m = Chem.MolFromMolFile(os.path.join(raw_data_path, ref_folder, ligand_name+'.sdf'))
        # ref_m = Chem.MolFromMolFile(os.path.join(raw_data_path, ref_folder, file))
        if gen_m == None or ref_m == None:
            continue
            
        gen_mols[ligand] = gen_m
        ref_mols[ligand] = ref_m

    return gen_mols, ref_mols

In [20]:
# gen_mols, ref_mols = get_mols(gen_path, ligand_name=None)
gen_mols, ref_mols = get_mols(gen_path, ligand_name=ligand_name)

100%|█████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 603.75it/s]


In [21]:
len(gen_mols.values())

500

In [22]:
unique_smiles = set([Chem.MolToSmiles(mol, canonical=True) for mol in gen_mols.values() if mol is not None])

In [23]:
unique_smiles

{'C=C1[C@@H](C(=CCCC)C(C)=O)C[C@@H](F)[C@H]1CCCC',
 'C=C1[C@@H](C(=CCCC)C(C)=O)C[C@@H](F)[C@H]1[C@@H](C)CC'}